In [1]:
# Third party packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import matplotlib.pyplot as plt
from io import StringIO
from IPython.display import display

# Adding path
from os.path import abspath
import sys
sys.path.insert(0, abspath('..'))

# 
from query_request import *
from query_def import *
import credentials
# from sqlalchemy import create_engine

# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth=None


In [2]:
# Demo for LDM

# Upload dataset
# downlaod dataset

In [3]:
# Demo for dragoman

# [Creating Endpoint](../configs/docker_command_icews.sh)

In [4]:
# create rdfmts
from DeTrusty import run_query
from DeTrusty.Molecule.MTCreation import create_rdfmts
from DeTrusty.Molecule.MTManager import ConfigFile

# Creating endpoints dictionary
endpoints = dictfrmjson('../configs/endpoints_sample.json')
print(endpoints)

# Creating RDF molecule template (rdfmts) from endpoints
# log_rdfmts = create_rdfmts(endpoints, '../configs/rdfmts.json')

# if already rdfmts has been created 
config_private = ConfigFile('../configs/rdfmts_private.json')
config_public = ConfigFile('../configs/rdfmts_public.json')

Total time taken by dictfrmjson function: 4.9591064453125e-05
{'https://labs.tib.eu/sdm/worldbank_endpoint/sparql': {}, 'https://labs.tib.eu/sdm/icews_endpoint/sparql': {}, 'https://labs.tib.eu/sdm/lei_endpoint/sparql': {}, 'https://skynet.coypu.org/coypu-internal': {'username': 'Your UserName', 'password': 'Your Password'}}


## Endpoints Status

In [5]:
# Demo query to all available enpoints

# demo query
query_test = """SELECT * WHERE{ ?Subject a ?Concept }LIMIT 1000"""


# creating request for cmemc
display("############### Cmemc Endpoint ###############")
cmemc_request = SPARQLRequest(client_url_imp, client_id_imp, client_secret_imp , 'oauth')
# executing query and printing response
cmemc_request.execute(query_test)
buf = StringIO(cmemc_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for skynet
display("############### Skynet Endpoint ###############")
skynet_request = SPARQLRequest(skynet_endpoint, skynet_user, skynet_pass, 'basic')
# executing query and printing response
skynet_request.execute(query_test)
buf = StringIO(skynet_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for worldbank
display("############### Worldbank Endpoint ###############")
worldbank_request = SPARQLRequest(worldbank_endpoint)
# executing query and printing response
worldbank_request.execute(query_test)
display(worldbank_request.response.status_code)
buf = StringIO(worldbank_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for LEI
display("############### LEI Endpoint ###############")
lei_request = SPARQLRequest(lei_endpoint)
# executing query and printing response
lei_request.execute(query_test)
buf = StringIO(lei_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))

# creating request for ICEWS
display("############### ICEWS Endpoint ###############")
icews_request = SPARQLRequest(icews_endpoint)
# executing query and printing response
icews_request.execute(query_test)
buf = StringIO(icews_request.response.content.decode('utf-8'), newline='\r\n')          
display(pd.read_csv(buf, encoding='utf-8').tail(5))


'############### Cmemc Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.40321803092956543


,Subject,Concept
995,https://vocab.eccenca.com/shacl/GraphTemplateJinjaTemplatePropertyShape,http://www.w3.org/2000/01/rdf-schema#Resource
996,https://vocab.eccenca.com/shacl/GraphTemplateJinjaTemplatePropertyShape,http://www.w3.org/ns/shacl#PropertyShape
997,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
998,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/2000/01/rdf-schema#Resource
999,https://vocab.eccenca.com/shui/jinjaTemplateString,http://www.w3.org/2002/07/owl#DatatypeProperty


'############### Skynet Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.06522750854492188


...

Passed Query Status: 200
Total time taken by execute function: 0.06196999549865723


,Subject,Concept
995,https://data.coypu.org/lei/2138006HHBEMF1TGWU29-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
996,https://data.coypu.org/lei/2138006HHUG8LC4UD387-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
997,https://data.coypu.org/lei/2138006HHXWLEK5WVK02-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
998,https://data.coypu.org/lei/2138006HI1P4JTM2WM22-LA,https://www.gleif.org/ontology/Base/PhysicalAddress
999,https://data.coypu.org/lei/2138006HI5AF5E1P4T95-LA,https://www.gleif.org/ontology/Base/PhysicalAddress


'############### ICEWS Endpoint ###############'

Passed Query Status: 200
Total time taken by execute function: 0.0616455078125


,Subject,Concept
995,https://data.coypu.org/icews/36616020,https://schema.coypu.org/global#Conflict
996,https://data.coypu.org/icews/36616021,https://schema.coypu.org/global#Conflict
997,https://data.coypu.org/icews/36616024,https://schema.coypu.org/global#Conflict
998,https://data.coypu.org/icews/36616025,https://schema.coypu.org/global#Conflict
999,https://data.coypu.org/icews/36616026,https://schema.coypu.org/global#Conflict


## Federated Query

In [6]:
# creating request for Federated Query
display("############### Federated Query ###############")
fdq_request = SPARQLRequest(is_fdq=True)
fdq_request.execute(query_test, config_public, False)
fdq_request.save('../query_results/', 'query_test')

'############### Federated Query ###############'

2022-11-30 17:11:18,649 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/lei_endpoint/sparql/
2022-11-30 17:11:18,662 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/worldbank_endpoint/sparql/
2022-11-30 17:11:18,665 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/icews_endpoint/sparql/


Total time taken by execute function: 0.6897144317626953
Index                    128
Subject.value           8000
Subject.type            8000
Concept.value           8000
Concept.type            8000
__meta__.is_verified    1000
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Subject.value         1000 non-null   object
 1   Subject.type          1000 non-null   object
 2   Concept.value         1000 non-null   object
 3   Concept.type          1000 non-null   object
 4   __meta__.is_verified  1000 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 32.4+ KB
None
Total time taken by json_to_csv function: 0.04625272750854492
Total time taken by save function: 0.04714608192443848


In [8]:
print(query_1_fdq_ex)
print('########################### EXAMPLE ##########################\n{}'.format(query_1_fdq_ex))
fdq_request.execute(query_1_fdq_ex, config_private, False)
fdq_request.save('../query_results/', 'query_1_fdq_ex')

PREFIX coy: <https://schema.coypu.org/global#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX time: <http://www.w3.org/2006/time#> 

PREFIX wb: <http://worldbank.org/>
PREFIX wbi: <http://worldbank.org/Indicator/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX geo: <https://www.geonames.org/ontology#>
SELECT ?country_code ?year ?year_dis ?value ?disaster
WHERE {
    ?indicator a wb:AnnualIndicatorEntry .
    ?indicator wb:hasIndicator <http://worldbank.org/Indicator/EN.ATM.CO2E.KT> .
    ?indicator wb:hasCountry ?country .
    ?indicator owl:hasValue ?value .
    ?indicator time:year ?year .
    ?country   dc:identifier ?country_code .

    ?disaster a coy:Disaster .
    ?disaster time:year ?year_dis .
    ?disaster geo:countryCode ?countr

2022-11-30 17:11:32,878 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://skynet.coypu.org/coypu-internal
2022-11-30 17:11:32,890 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://skynet.coypu.org/coypu-internal
2022-11-30 17:11:32,892 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/worldbank_endpoint/sparql
2022-11-30 17:11:32,916 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://labs.tib.eu/sdm/worldbank_endpoint/sparql
2022-11-30 17:11:33,623 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://skynet.coypu.org/coypu-internal
2022-11-30 17:11:33,625 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://skynet.coypu.org/coypu-internal
2022-11-30 17:11:33,641 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://skynet.coypu.org/coypu-internal
2022-11-30 17:11:33,685 - DeTrusty.Wrapper.RDFWrapper - INFO - Contacting endpoint: https://skynet.coypu.org/coy

Total time taken by execute function: 27.11950159072876
Index                       128
country_code.value      1560960
country_code.type       1560960
year.value              1560960
year.type               1560960
year_dis.value          1560960
year_dis.type           1560960
value.value             1560960
value.type              1560960
disaster.value          1560960
disaster.type           1560960
__meta__.is_verified     195120
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195120 entries, 0 to 195119
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   country_code.value    195120 non-null  object
 1   country_code.type     195120 non-null  object
 2   year.value            195120 non-null  object
 3   year.type             195120 non-null  object
 4   year_dis.value        195120 non-null  object
 5   year_dis.type         195120 non-null  object
 6   value.value           1

In [ ]:
def carbon_emission_group(carbon_emission):
    if carbon_emission/1000000 < 0.25:
        return '0.00-0.25'
    elif carbon_emission/1000000 >= 0.25 and carbon_emission/1000000 < 0.5:
        return '0.25-0.5'
    elif carbon_emission/1000000 >= 0.5 and carbon_emission/1000000 < 0.75:
        return '0.50-0.75'
    else:
        return '> 0.75'
    

df = pd.read_csv('../query_results/query_1_fdq_ex.csv', encoding='utf-8')
df['carbon_emission(1e6)'] = df['carbon_emission'].apply(carbon_emission_group)

df.head(10)

In [ ]:

fig = px.scatter_geo(df, locations="country_code", color="carbon_emission(1e6)",
                     hover_data=['carbon_emission'], size="no_of_disasters",
                     animation_frame="year",
                     projection="equirectangular", scope='europe')
fig.update_layout(title_text='Carbon emission and the number of disaster events for various countries across years',\
                  title_font_color="red", title_font_family="Times New Roman", title_font_size=25)
fig.write_html('../../query_results/query_1_fdq_ex_sql_new.html')
fig.show()